In [1]:
# Global imports
import os
import sys
import pathlib
import time

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn

from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataset import random_split
import torch.optim as optim

from sklearn.preprocessing import MinMaxScaler

In [2]:
cwd = pathlib.Path().resolve()
src = cwd.parent
root = src.parent
sys.path.append(str(src))

In [3]:
#initialize GPU -  In case of windows use cuda instead of nps
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("Is CUDA enabled?",torch.cuda.is_available())
print("Number of GPUs",torch.cuda.device_count())

Is CUDA enabled? False
Number of GPUs 0


###  Simple RNN
Try Simple RNN with CNN later

In [20]:
from models.Networks import SimpleRNN
from models.Networks import CNN

node_features = 2 # DEM and WD

model_CNN = CNN(node_features=node_features,
                n_downsamples=4,
                initial_hid_dim=32,
                batch_norm=True,
                bias=True).to(device)


In [9]:
from utils.simulation import Simulation
from train import train_and_validate, evaluate_model

In [48]:
def loading():
    sims = Simulation.load_simulations(str(root)+"/data/processed_data/normalized_training_data", sim_amount = 1, number_grids=64)

    n_timesteps = 96
    grid_size = 64
    channels = 2   # water depth and topography

    # reformat the data
    X = np.zeros((n_timesteps*len(sims), grid_size, grid_size, channels))   # timestep * grid_x * grid_y * channels
    Y = np.zeros(X[:,:,:,0].shape)   # timestep * grid_x * grid_y

    for i in range(len(sims)):   # number of simulations loaded in for training/validation

        sim = sims[i]            # get simulation
        topography = sim.topography

        for t_i in range(n_timesteps):    # number of timesteps

            wd, vx, vy = sim.return_timestep(t_i)
            X[t_i+i*n_timesteps, :, :, 0] = wd
            X[t_i+i*n_timesteps, :, :, 1] = topography

            wd, vx, vy = sim.return_timestep(t_i+1)
            Y[t_i+i*n_timesteps, :, :] = wd

    return(X,Y)

In [49]:
X,Y = loading()
display(X.shape) # input (DEM, WD) - one simulation
display(Y.shape) # output (WD) - one simulation

(96, 64, 64, 2)

(96, 64, 64)

In [73]:
sims = Simulation.load_simulations(str(root)+"/data/processed_data/normalized_training_data", sim_amount = 2, number_grids=64)
display(sims[1].wd)

array([[[0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
        [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
        [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
        ...,
        [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
        [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
        [0.0025, 0.    , 0.    , ..., 0.    , 0.    , 0.    ]],

       [[0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
        [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
        [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
        ...,
        [0.034 , 0.0395, 0.0428, ..., 0.    , 0.    , 0.    ],
        [0.0528, 0.0577, 0.0581, ..., 0.    , 0.    , 0.    ],
        [0.0781, 0.0595, 0.0539, ..., 0.    , 0.    , 0.    ]],

       [[0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
        [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
        [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
        ...,
        [0.0

In [ ]:
#create datasets and data loaders
train_dataset = TensorDataset(torch.tensor(X_tra, dtype=torch.float32), torch.tensor(Y_tra, dtype=torch.float32))
val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(Y_val, dtype=torch.float32))

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# defining the optimizer
optimizer = optimizer(model.parameters(), lr=lr)

# defining the save path
save_path = "../results/trained_models/" + model_name

# training
train_losses, val_losses, best_val_loss, time = train_and_validate(model, train_loader, val_loader, criterion, optimizer, num_epochs, device, save_path)

# Load the best model
    model.load_state_dict(torch.load(save_path))

In [19]:
train_rnn(model = SimpleRNN,
          sim_amount=1,
          training_size=0.8,
          batch_size=4,
          num_epochs = 200,
          lr = 0.0005,
          criterion = nn.MSELoss(),
          optimizer = optim.AdamW,
          model_name = 'babies_first_RNN')

X_tra.shape:  (76, 64, 64, 2)
Y_tra.shape:  (76, 64, 64)
X_val.shape:  (20, 64, 64, 2)
Y_val.shape:  (20, 64, 64)


TypeError: parameters() missing 1 required positional argument: 'self'

In [43]:
simulation_length = 10 # number of simulation
grid_size = 64 # number of grid
timestep = 96 # number of time step

# Create a sample 2D array
input_data = torch.rand((sequence_length, input_size))

# Reshape into a 3D tensor (simulation_length, timestep, grid_size)
input_data = input_data.unsqueeze(1).expand(-1, batch_size, -1)

# Create an RNN layer
rnn = nn.RNN(input_size, hidden_size=10, num_layers=1, batch_first=True)


In [45]:
display(input_data.shape)

torch.Size([10, 96, 64])